In [3]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to data with images
data_dir = 'Data'
train_dir = 'TrainData'
val_dir = 'ValData'

# Creation of directories for train and validation data if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Function to automatically split images into train and validation directories
def split_data_into_train_val(data_dir, train_dir, val_dir, split_ratio=0.8):
    classes = os.listdir(data_dir)
    for cls in classes:
        cls_dir = os.path.join(data_dir, cls)
        if os.path.isdir(cls_dir):
            images = os.listdir(cls_dir)
            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            val_images = images[split_index:]
            for img in train_images:
                src = os.path.join(cls_dir, img)
                dst = os.path.join(train_dir, cls)
                os.makedirs(dst, exist_ok=True)
                shutil.copy(src, dst)
            for img in val_images:
                src = os.path.join(cls_dir, img)
                dst = os.path.join(val_dir, cls)
                os.makedirs(dst, exist_ok=True)
                shutil.copy(src, dst)

# Splitting data into train and validation directories
split_data_into_train_val(data_dir, train_dir, val_dir)

# Defining image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 16

# Creati data generators with augmentation for training and validation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Loading bas model VGG16 
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freezing convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Creating new classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Model compiling
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

loss, accuracy = model.evaluate(val_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

model.save('fruit_classification_model.keras')

Found 926 images belonging to 30 classes.
Found 373 images belonging to 30 classes.
Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.1053 - loss: 7.6629 - val_accuracy: 0.3940 - val_loss: 2.2886
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3750 - loss: 2.4646 - val_accuracy: 0.6000 - val_loss: 2.1727
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.2614 - loss: 2.5885 - val_accuracy: 0.5924 - val_loss: 1.6084
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3125 - loss: 2.2398 - val_accuracy: 0.8000 - val_loss: 1.8595
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.4056 - loss: 2.0533 - val_accuracy: 0.7065 - val_loss: 1.4170
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3750 - loss: 2.1677 - val_accuracy: 0.6000 - val_loss: 1.3754
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.4870 - loss: 1.7609 - val_accuracy: 0.7418 - val_loss: 1.1276
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━